# Image classificatin with Transfer Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import os

## Data Preparation

In [ ]:
!wget -q https://raw.githubusercontent.com/anshupandey/Computer-Vision/master/image_data.zip
!unzip image_data.zip

Archive:  image_data.zip
  inflating: image_data/car3.png     
  inflating: image_data/cat.jpeg     
  inflating: image_data/cat2.jpg     
   creating: image_data/data/
   creating: image_data/data/dogs/
  inflating: image_data/data/dogs/dog.1.jpg  
  inflating: image_data/data/dogs/dog.10.jpg  
  inflating: image_data/data/dogs/dog.100.jpg  
  inflating: image_data/data/dogs/dog.101.jpg  
  inflating: image_data/data/dogs/dog.102.jpg  
  inflating: image_data/data/dogs/dog.103.jpg  
  inflating: image_data/data/dogs/dog.104.jpg  
  inflating: image_data/data/dogs/dog.105.jpg  
  inflating: image_data/data/dogs/dog.106.jpg  
  inflating: image_data/data/dogs/dog.107.jpg  
  inflating: image_data/data/dogs/dog.108.jpg  
  inflating: image_data/data/dogs/dog.109.jpg  
  inflating: image_data/data/dogs/dog.11.jpg  
  inflating: image_data/data/dogs/dog.110.jpg  
  inflating: image_data/data/dogs/dog.111.jpg  
  inflating: image_data/data/dogs/dog.112.jpg  
  inflating: image_data/data/dog

In [ ]:
main_folder = r"/content/image_data/data/"
cnames = os.listdir(main_folder)
cnames

['horses', 'dogs', 'Humans']

In [ ]:
from tensorflow.keras.applications import resnet50

In [ ]:
x = []
y = []

for  folder in cnames:
    # get the list of all image names
    imglist = os.listdir(main_folder+folder)
    for imgname in imglist:
        img = image.load_img(main_folder+folder+"/"+imgname,target_size=(150,150))
        img = image.img_to_array(img)
        img = resnet50.preprocess_input(img) # preprocessing image using resnet50 preprocessor
        x.append(img)
        y.append(cnames.index(folder))

In [ ]:
x = np.array(x)
print(x.shape)
y = to_categorical(y)# onehot encoding the labels
print(y.shape)

(606, 150, 150, 3)
(606, 3)


## Modelling the CNN

In [ ]:
from tensorflow.keras import models,layers

In [ ]:
#load the pretrained model
resmodel = resnet50.ResNet50(weights='imagenet')
resmodel.summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [ ]:
input_layer = layers.Input(shape=(150,150,3))

resmodel = resnet50.ResNet50(weights='imagenet',input_tensor=input_layer,include_top=False)
resmodel.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
__________________________

In [ ]:
#access the current last layer
current_last_layer = resmodel.output
ft = layers.Flatten()(current_last_layer)
output_layer = layers.Dense(3,activation='softmax')(ft)
model = models.Model(inputs=input_layer,outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
for layer in model.layers[:-1]:
  layer.trainable=False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(x,y,test_size=0.2,random_state=5)
print(xtr.shape,xts.shape)
print(ytr.shape,yts.shape)

(484, 150, 150, 3) (122, 150, 150, 3)
(484, 3) (122, 3)


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',
             metrics=['accuracy'])
model.fit(xtr,ytr,epochs=5,batch_size=32,validation_data=(xts,yts))

Epoch 1/5
16/16 [==============================] - 36s 187ms/step - loss: 1.5490 - accuracy: 0.8574 - val_loss: 0.5462 - val_accuracy: 0.9590
Epoch 2/5
16/16 [==============================] - 1s 72ms/step - loss: 0.0437 - accuracy: 0.9917 - val_loss: 0.6976 - val_accuracy: 0.9672
Epoch 3/5
16/16 [==============================] - 1s 71ms/step - loss: 0.0051 - accuracy: 0.9979 - val_loss: 0.7087 - val_accuracy: 0.9754
Epoch 4/5
16/16 [==============================] - 1s 72ms/step - loss: 3.2928e-07 - accuracy: 1.0000 - val_loss: 0.8925 - val_accuracy: 0.9672
Epoch 5/5
16/16 [==============================] - 1s 71ms/step - loss: 7.2306e-05 - accuracy: 1.0000 - val_loss: 0.9244 - val_accuracy: 0.9672


In [ ]:
!wget -q https://res.cloudinary.com/dk-find-out/image/upload/q_80,w_1920,f_auto/MA_00600259_vpf1m0.jpg


In [ ]:
img = image.img_to_array(image.load_img("MA_00600259_vpf1m0.jpg",target_size=(150,150)))
model.predict(img.reshape(1,150,150,3))

array([[1.000000e+00, 5.631269e-18, 7.502188e-21]], dtype=float32)

In [ ]:
cnames

['horses', 'dogs', 'Humans']

In [ ]:
img.shape

(150, 150, 3)